# Dynamic Pricing Algorithm Based on EDA Insights by Anusha N

Inputs Taken:
For each event, the model considers:

Current price

Number of days left until the event

Current number of bookings

Maximum seats (capacity)

Class popularity (if any)

Location factor (if any)

Calculate Fill Rate:
Fill Rate = current bookings ÷ max capacity
This indicates how full the event is.

Calculate Time Until Event:
Helps determine urgency—early bird vs. last-minute.

Apply Pricing Rules:
Based on business insights, rules adjust the price up or down:

Early bookings: slight discounts to incentivize

Near-capacity or popular events: price increases

Low demand close to event: discounts to fill seats

High fill rate + short time = premium pricing

Adjust Price:
The model adds/subtracts a % of the current price based on combined rule effects.

Apply Bounds:
Final price is clipped between 70% and 150% of the base price to avoid extremes.

Output:
A dynamically adjusted price for each event, tailored to its real-time booking and timing context.

In [12]:
import pandas as pd
import numpy as np
from datetime import datetime

In [14]:
# Load and Initial Data Preparation ---
df=pd.read_excel("edaoutput.xlsx",index_col=0)

In [15]:
# --- Utility Functions ---
def get_fill_rate(current_bookings, max_bookees):
    return current_bookings / max_bookees if max_bookees > 0 else 0

def get_days_to_event(activity_date, today):
    return (activity_date - today).days

# --- Simplified Dynamic Pricing Algorithm ---
def dynamic_price_adjustment(row, today=pd.Timestamp("2018-06-01")):
    base_price = row['Price (INR)']
    max_bookees = row['MaxBookees']
    current_bookings = row['Number Booked']
    booking_date = row['BookingEndDateTime (Month / Day / Year)']
    price_tier = row['Price_Tier']
    weekday = row['Booking_Weekday']

    fill_rate = get_fill_rate(current_bookings, max_bookees)
    days_left = get_days_to_event(booking_date, today)

    adjustment = 0.0

    # Time-Based Rules
    if days_left > 30:
        adjustment -= 0.05  # early bird
    elif days_left < 3 and fill_rate < 0.5:
        adjustment -= 0.10  # last minute, low demand
    elif days_left < 7 and fill_rate > 0.8:
        adjustment += 0.15  # urgent demand

    # Fill Rate Rules
    if fill_rate >= 0.95:
        adjustment += 0.20  # premium
    elif fill_rate <= 0.10 and days_left > 7:
        adjustment -= 0.07  # stimulate demand

    # Tier or Day-based Pricing
    if price_tier == "Low" and weekday in ["Saturday", "Sunday"]:
        adjustment += 0.05  # high weekend demand for low-price events
    elif price_tier == "High" and fill_rate < 0.3:
        adjustment -= 0.08  # high price but low interest

    # Final new price, bounded
    new_price = base_price * (1 + adjustment)
    new_price = max(base_price * 0.7, min(new_price, base_price * 1.5))
    return round(new_price, 2)

In [18]:
# --- Apply to Dataset ---
df['Dynamic_Price'] = df.apply(dynamic_price_adjustment, axis=1)

# Optional: View top results
print(df[['Price (INR)', 'Dynamic_Price', 'Fill_Rate', 'Price_Tier', 'Booking_Weekday']].head())

   Price (INR)  Dynamic_Price  Fill_Rate Price_Tier Booking_Weekday
0          499         474.05       0.48        Low          Sunday
1          499         523.95       0.60        Low          Sunday
2          499         523.95       0.56        Low          Sunday
3          499         474.05       0.36        Low          Sunday
4          499         474.05       0.28        Low          Sunday
